# ETL PI MLOps

In [ ]:
# Se importan las librerias a usar 
import pandas as pd
import numpy as np

In [ ]:
# Se crea el dataset, a partir de los csv
df_peliculas = pd.read_csv('movies_dataset.csv')
df_creditos = pd.read_csv('credits.csv')

# Transformación

1.- Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

In [ ]:
# Desanidamos belongs to collection
# Se rellenar valores vacios para poder trabajar mejor
df_peliculas['belongs_to_collection'] = df_peliculas['belongs_to_collection'].fillna('{}')
df_peliculas.reset_index(drop=True, inplace=True) 
# Se estructura y organizan los datos desanidados
df_collection = pd.json_normalize(df_peliculas['belongs_to_collection'].map(eval))

# Renombramos la columna 'name' como 'belongs_to_collection'
df_collection.rename(columns={'name': 'belongs_to_collection'}, inplace=True)

# Obtenemos solo la columna belongs_to_collection
df_collection = df_collection['belongs_to_collection'].copy()

# Visualimos el nuevo DataFrame
df_collection

In [ ]:
# Desanidamos genres
# Se estructura y organizan los datos desanidados
df_genres = pd.json_normalize(df_peliculas['genres'].map(eval))

# Se obtiene la columna de genres a partir de los datos de name
df_genres['genres'] = df_genres.apply(lambda x: ','.join([genre['name'] for genre in x if pd.notna(genre)]), axis=1)

# Se crea un nuevo DataFrame solo con la columna de genres
df_genres = df_genres[['genres']].copy()

# Visualizamos el nuevo DataFrame
df_genres

In [ ]:
# Desanidamos production companies
# Se estructura y organizan los datos desanidados
df_companies = pd.json_normalize(df_peliculas['production_companies'].map(eval))

# Se obtiene la columna de production countries
df_companies['production_companies'] = df_companies.apply(lambda x: ','.join([country['name'] for country in x if pd.notna(country)]), axis=1)

# Se crea un nuevo DataFrame solo con la columna
df_companies = df_companies[['production_companies']].copy()

# Visualizamos el nuevo DataFrame
df_companies

In [ ]:
# Desanidamos production countries
# Se estructura y organizan los datos desanidados
df_countries = pd.json_normalize(df_peliculas['production_countries'].map(eval))

# Se obtiene la columna de production countries
df_countries['production_countries'] = df_countries.apply(lambda x: ','.join([country['name'] for country in x if pd.notna(country)]), axis=1)

# Se crea un nuevo DataFrame solo con la columna
df_countries = df_countries[['production_countries']].copy()

# Visualizamos el nuevo DataFrame
df_countries

In [ ]:
# Desanidamos spoken languages
# Se estructura y organizan los datos desanidados
df_languages = pd.json_normalize(df_peliculas['spoken_languages'].map(eval))

# Se obtiene la columna de spoken languages
df_languages['spoken_languages'] = df_languages.apply(lambda x: ','.join([country['name'] for country in x if pd.notna(country)]), axis=1)

# Se crea un nuevo DataFrame solo con la columna
df_languages = df_languages[['spoken_languages']].copy()

# Visualizamos el nuevo DataFrame
df_languages

In [ ]:
# Eliminamos las columnas anidadas, para posterior sustituirlas por las mismas columnas ya desanidadas
df_peliculas = df_peliculas.drop(columns=['belongs_to_collection','spoken_languages', 'production_countries', 'genres', 'production_companies'])
df_completo = pd.concat([df_peliculas, df_collection, df_genres, df_companies, df_countries, df_languages], axis=1)
df_completo

2.- Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [ ]:
# Se rellenan los valores nulos
df_completo['budget'] = df_completo['budget'].fillna(0)
df_completo['revenue'] = df_completo['revenue'].fillna(0)

3.- Los valores nulos del campo release date deben eliminarse.

In [ ]:
# Contamos los datos nulos en release date
df_completo['release_date'].isnull().sum()

In [ ]:
# Eliminamos los datos nulos y contamos nuevamente para asegurarnos que fueron eliminados
df_completo = df_completo.dropna(subset=['release_date'])
df_completo['release_date'].isnull().sum()

4.- De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [ ]:
# Se convierte la columna de fechas a tipo datetime
df_completo['release_date'] = pd.to_datetime(df_completo['release_date'], errors='coerce')

# Se extrae el año de la fecha de lanzamiento
df_completo['release_year'] = df_completo['release_date'].dt.year

# Verificamos que se haya agregado correctamente
df_completo

5.- Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [ ]:
# Se convierten las columnas 'revenue' y 'budget' a tipo numérico
df_completo['revenue'] = pd.to_numeric(df_completo['revenue'], errors='coerce')
df_completo['budget'] = pd.to_numeric(df_completo['budget'], errors='coerce')

# Se crea la columna 'return' y se calcula el retorno de inversión
df_completo['return'] = df_completo['revenue'].div(df_completo['budget'], fill_value=0)

# Se establece 0 en los casos donde budget sea 0 o haya valores faltantes en revenue o budget
faltantes = (df_completo['budget'] == 0) | (df_completo[['revenue', 'budget']].isnull().any(axis=1))
df_completo.loc[faltantes, 'return'] = 0

# Se visualiza para asegurarse que los datos estan correctos
df_completo

6.- Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [ ]:
# Se eliminan las columnas solicitadas
df_completo = df_completo.drop(columns = ['video','imdb_id','adult','original_title','poster_path','homepage'])

# Se comprueba que ya no esten
df_completo.info() 

In [ ]:
df_completo.to_csv('datos_completos.csv', index=False)